In [1]:
import altair as alt
import datapane as dp
import sys, yaml, psycopg2, os
import pandas as pd
import numpy as np
import mysql.connector
from sqlalchemy import create_engine
import plotly.express as px

In [2]:
dp.login(token="10867dc863351170ba6ebe5b6eb4feff13655d69")

Connected successfully to https://cloud.datapane.com as miguel.angel.velazco.mav@gmail.com


'miguel.angel.velazco.mav@gmail.com'

In [3]:
def read_yaml_file(yaml_file):
    """ load yaml cofigurations """

    config = None
    try:
        with open(yaml_file, 'r') as f:
            config = yaml.safe_load(f)
    except:
        raise FileNotFoundError('Couldnt load the file')

    return config


def get_db_conn_mysql(creds_file):
    """ Get an authenticated psycopg db connection, given a credentials file"""
    creds = read_yaml_file(creds_file)['db']

    connection = mysql.connector.connect(
        user=creds['user'],
        password=creds['pass'],
        host=creds['host'],
        port=creds['port'],
        database=creds['db']
    )

    return connection

In [4]:
db_conn_mysql = get_db_conn_mysql("key_mysql.yaml")

In [5]:
q = """
    select 
        * 
    from 
        bp.cre_prices
    where 
        place_id in (11702)
    order by 
        update_data_date desc;
"""

In [6]:
df = pd.read_sql(q, db_conn_mysql)

C:\Users\migue\AppData\Local\Temp/ipykernel_16024/94881800.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(q, db_conn_mysql)


In [7]:
df.columns

Index(['index_', 'place_id', 'regular', 'update_date_regular', 'premium',
       'update_date_premium', 'diesel', 'update_date_diesel',
       'update_data_date'],
      dtype='object')

In [8]:
columns2drop = ['index_', 'place_id', 'update_date_regular', 'update_date_premium', 'update_date_diesel']
df.drop(columns = columns2drop, inplace = True)

In [9]:
df['king'] = df['premium'] + 7

In [10]:
df.rename(columns = {'regular' : 'individual', 'premium' : 'matromonial', 'diesel' : 'queen'}, inplace = True)

In [11]:
sizes = ['individual', 'matromonial', 'queen', 'king']
df_f = pd.DataFrame()

for size in sizes:
    df_buffer = df[[size, 'update_data_date']]
    df_buffer['size'] = size
    df_buffer.rename(columns = {size : 'price'}, inplace = True)    
    df_f = pd.concat([df_f, df_buffer], ignore_index = True)

C:\Users\migue\AppData\Local\Temp/ipykernel_16024/4082525127.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_buffer['size'] = size
C:\Users\migue\AppData\Local\Temp/ipykernel_16024/4082525127.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_buffer.rename(columns = {size : 'price'}, inplace = True)
C:\Users\migue\AppData\Local\Temp/ipykernel_16024/4082525127.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

In [15]:
plot1 = (
    px.line(df_f, x='update_data_date', y='price', color='size', labels = {'update_data_date' : 'date'})
)

dp.upload_report(
    [
        dp.Text('The next one is an example dashboard for prices (the data is an example from another business)'),
        dp.Plot(plot1),
        dp.DataTable(df_f)
    ],
    name="Zebrands example report"
)

Uploading report and associated data - *please wait...*

Report successfully uploaded - view and share at <a href='https://cloud.datapane.com/reports/E7ooVo7/zebrands-example-report/' target='_blank'>here</a>.

Uploaded CloudReport - view at https://cloud.datapane.com/reports/E7ooVo7/zebrands-example-report/